In [0]:
import cv2
import numpy as np
import pandas as pd
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import os
import PIL
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [0]:
''' Load the facial keypoints data. 
    Return: training face images with corresponding key points. ''' 
def load_data():
    # load & read data
    fname = 'data/training.csv'
    df = pd.read_csv(os.path.expanduser(fname))  
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    # drop missing values
    df = df.dropna() 

    # normalize
    X = np.vstack(df['Image'].values) / 255.  
    X = X.astype(np.float32)
    
    # reshape (96, 96, 1)
    X = X.reshape(-1, 96, 96, 1) 

    y = df[df.columns[:-1]].values

    # normalize
    y = (y - 48) / 48  

    X, y = shuffle(X, y, random_state=42)  
    y = y.astype(np.float32)

    return X, y


''' This class builds a simple convolutional neural network. 
    Predicts facial keypoints, given an image of a face. '''
class CNNModel:
    def __init__(self, filename):
        # load model if exists
        # else, train new model
        self.filename = filename
        if os.path.exists(self.filename):
            self.model = load_model(self.filename)
        else:
            self.build_model()
    
    ''' Constructs a new convolutional neural network, and
        saves as a .h5 file. '''
    def build_model(self): 
        self.model = Sequential()
        self.model.add(Convolution2D(32, (5, 5), input_shape=(96,96,1), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Convolution2D(64, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.1))

        self.model.add(Convolution2D(128, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.2))

        self.model.add(Convolution2D(30, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.3))

        self.model.add(Flatten())

        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(256, activation='relu'))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(30))

        self.model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

        x_train, y_train = load_data()
        self.model.fit(x_train, y_train, epochs=100, batch_size=200, verbose=1, validation_split=0.2)
        
        self.model.save(self.filename)

    ''' Predict keypoints, given a face image. '''
    def predict(self, face): 
        return self.model.predict(face)


In [0]:
def cv2_imshow(a):
  """A replacement for cv2.imshow() for use in Jupyter notebooks."""
  a = a.clip(0, 255).astype('uint8')
  # cv2 stores colors as BGR; convert to RGB
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

In [0]:
''' Constructs a filter, given an image. 
    You can overlay this filter on the eyes, mouth, forehead, or nose. '''
class Filter: 
    def __init__(self, filepath):
        # read in filter image given filepath
        self.filter = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)

    def eyes_overlay(self, points, frame):
        # resize the filter image w.r.t. points
        f_width = int(points[7][0] - points[9][0])
        f_height = int(points[10][1] - points[8][1])
        f_resized = cv2.resize(self.filter, (f_width, f_height), interpolation=cv2.INTER_CUBIC)

        # get transparent region (i.e. alpha > 0.7)
        transparent_region = f_resized[:, :, 3] > 0.7 

        # get boundaries (global coord.)
        xmin, xmax, ymin, ymax = self.get_x_y_min_max(points[9], f_width, f_height)
        frame[ymin:ymax, xmin:xmax, :][transparent_region] = f_resized[:, :, :3][transparent_region]

    def mouth_overlay(self, points, frame):
        # TODO: create overlay function for mouth with any image found online!
        return None

    def forehead_overlay(self, points, frame):
        # TODO: create overlay for forehead images (i.e. dog ears or flower crowns!)
        return None

    def nose_overlay(self, points, frame, vertical_shift=0):
        # resize the filter image w.r.t. points
        f_width = 100
        f_height = 50
        f_resized = cv2.resize(self.filter, (f_width, f_height), interpolation=cv2.INTER_CUBIC)

        # get transparent region (i.e. alpha > 0.7)
        transparent_region = f_resized[:, :, 3] > 0.7 
        
        # get boundaries (global coord.)
        xmin, xmax, ymin, ymax = self.get_x_y_min_max(points[10], f_width, f_height)
        
        # shift the filter to be on center with the nose
        xmin -= f_width // 2
        xmax -= f_width // 2
        ymin += vertical_shift
        ymax += vertical_shift
        frame[ymin:ymax, xmin:xmax, :][transparent_region] = f_resized[:, :, :3][transparent_region]

    ''' 
    Returns the global coordinates box of the filter with respect to a certain point
    ''' 
    def get_x_y_min_max(self, boundary_point, filter_width, filter_height):
        ymin = int(boundary_point[1])
        ymax = ymin + filter_height
        xmin = int(boundary_point[0])
        xmax = xmin + filter_width
        return xmin, xmax, ymin, ymax

In [0]:
# Load the model built in the previous step
my_model = CNNModel('my_model.h5')

# TODO: Face cascade to detect faces
face_cascade = ...

# TODO: add more filters!
moustache = Filter(os.path.join('filters', 'moustache.png'))

# Grab the current paintWindow
frame = cv2.imread("imagepath") # TODO: change to image path
frame = cv2.flip(frame, 1)
frame2 = np.copy(frame)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# TODO: Detect faces
faces = ...
if len(faces) == 0:
    cv2.putText(frame, "Finding Face...", (37, 37), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Selfie Filters', frame)

for (x, y, w, h) in faces:
    # Grab the face
    gray_face = gray[y:y+h, x:x+w]
    color_face = frame[y:y+h, x:x+w]

    # Normalize to match the input format of the model - Range of pixel to [0, 1]
    gray_normalized = gray_face / 255

    # Resize it to 96x96 to match the input format of the model
    original_shape = gray_face.shape
    face_resized = cv2.resize(gray_normalized, (96, 96), interpolation = cv2.INTER_AREA)
    face_resized_copy = face_resized.copy()
    face_resized = face_resized.reshape(1, 96, 96, 1)

    # Predicting the keypoints using the model
    keypoints = my_model.predict(face_resized)

    # De-Normalize the keypoints values
    keypoints = keypoints * 48 + 48

    # Map the Keypoints back to the original image
    face_resized_color = cv2.resize(color_face, (96, 96), interpolation = cv2.INTER_AREA)
    face_resized_color2 = np.copy(face_resized_color)

    x_scale = original_shape[1] / 96
    y_scale = original_shape[0] / 96

    # Pair them together
    points = []
    for i, co in enumerate(keypoints[0][0::2]):
        points.append((co, keypoints[0][1::2][i]))

    # Transform points to global coordinates
    for i in range(len(points)):
        points[i] *= np.array([x_scale, y_scale])
        points[i] += np.array([x, y])

    # TODO: Add more FILTERS to the frame
    moustache.nose_overlay(points, frame, -5)

    # Add KEYPOINTS to the frame2
    for keypoint in points:
        (px, py) = np.array(keypoint, dtype=np.int)
        cv2.circle(frame2, (px, py), 1, (0,255,0), 1)

    # Show the frame and the frame2
    cv2_imshow(frame)
    cv2_imshow(frame2)